<a href="https://colab.research.google.com/github/krishnabalajiwork/Ai-Hydroponics/blob/main/RandomForestRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta
import pytz

def calculate_days_to_maturity(age, ph):
    """
    Calculate days to maturity based on biological research findings
    - Optimal pH: 6.0-6.5 (fastest growth)
    - pH effects: Growth slows outside optimal range
    - Total lifecycle: 30-42 days depending on conditions
    """
    base_maturity = 35  # Base maturity at optimal conditions

    # pH effect (parabolic - slower growth away from optimal 6.2)
    ph_penalty = 8 * (ph - 6.2)**2  # Quadratic penalty

    # Age effect - younger plants need more days
    age_factor = max(0, (25 - age) * 0.2)  # Young plants grow slower

    total_days_to_mature = base_maturity + ph_penalty + age_factor
    days_remaining = max(0, min(total_days_to_mature - age, 42 - age))

    return days_remaining

# Create biologically accurate dataset
plant_ages = []
ph_levels = []
days_to_maturity = []

# Generate data for ages 1-40 and pH 5.0-8.0
for age in range(1, 41):
    for ph in np.arange(5.0, 8.1, 0.1):
        plant_ages.append(age)
        ph_levels.append(round(ph, 1))
        days_to_maturity.append(calculate_days_to_maturity(age, ph))

# Create DataFrame
df = pd.DataFrame({
    'Plant Age (Days)': plant_ages,
    'pH Level': ph_levels,
    'Days to Maturity': days_to_maturity
})

# Train Random Forest model for better non-linear relationship capture
X = df[['Plant Age (Days)', 'pH Level']]
y = df['Days to Maturity']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

def predict_fenugreek_harvest(plant_age, ph_level):
    """
    Predict fenugreek harvest with biologically accurate pH sensitivity
    """
    if plant_age < 1:
        return "❌ Invalid input: Plant Age must be at least 1 day."
    if not (5.0 <= ph_level <= 8.0):
        return "⚠️ pH outside tested range (5.0-8.0). Results may be inaccurate."

    # Handle mature plants
    if plant_age >= 42:
        return f"🌿 Plant age {plant_age} days: Beyond normal maturity. Ready for harvest."
    if plant_age >= 35 and ph_level >= 6.0 and ph_level <= 7.0:
        return f"🌿 Plant age {plant_age} days with optimal pH: Ready for harvest."

    # Predict using model
    input_data = pd.DataFrame({'Plant Age (Days)': [plant_age], 'pH Level': [ph_level]})
    predicted_days = model.predict(input_data)[0]

    # Ensure biological constraints
    days_to_harvest = max(0, min(predicted_days, 42 - plant_age))
    days_to_harvest = round(days_to_harvest, 1)

    expected_harvest_day = plant_age + days_to_harvest

    # Calculate growth rate based on conditions (optimal at pH 6.0-6.5)
    ph_efficiency = 1.0 - 0.15 * abs(ph_level - 6.2)  # Efficiency decreases away from optimal
    growth_rate = (20 * ph_efficiency) / (plant_age + days_to_harvest)

    # Current date and harvest prediction
    india_tz = pytz.timezone("Asia/Kolkata")
    current_date = datetime.now(india_tz)
    harvest_date = current_date + timedelta(days=days_to_harvest)

    # pH condition assessment
    if 6.0 <= ph_level <= 6.5:
        ph_status = "Optimal"
    elif 5.8 <= ph_level <= 7.0:
        ph_status = "Good"
    elif 5.5 <= ph_level <= 7.5:
        ph_status = "Suboptimal"
    else:
        ph_status = "Poor"

    return {
        "📅 Current Date": current_date.strftime('%A, %B %d, %Y, %I:%M %p %Z'),
        "🌱 pH Condition": f"{ph_status} (Optimal: 6.0-6.5)",
        "✅ Predicted Growth Rate": f"{growth_rate:.4f} cm/day",
        "⏳ Estimated Days to Harvest": f"{days_to_harvest} days",
        "📅 Expected Harvest Day": f"Day {expected_harvest_day:.1f}",
        "📆 Expected Harvest Date": harvest_date.strftime('%A, %B %d, %Y')
    }

# User input section
if __name__ == "__main__":
    try:
        plant_age = float(input("Enter Plant Age (Days): "))
        ph_level = float(input("Enter pH Level (5.0 - 8.0): "))

        result = predict_fenugreek_harvest(plant_age, ph_level)

        if isinstance(result, dict):
            for key, value in result.items():
                print(f"{key}: {value}")
        else:
            print(result)
    except ValueError:
        print("❌ Please enter numeric values for Plant Age and pH Level.")

Enter Plant Age (Days): 23
Enter pH Level (5.0 - 8.0): 6
📅 Current Date: Wednesday, July 23, 2025, 07:38 PM IST
🌱 pH Condition: Optimal (Optimal: 6.0-6.5)
✅ Predicted Growth Rate: 0.5419 cm/day
⏳ Estimated Days to Harvest: 12.8 days
📅 Expected Harvest Day: Day 35.8
📆 Expected Harvest Date: Tuesday, August 05, 2025
